- https://www.kaggle.com/datasets/amitabhajoy/bengaluru-house-price-data 

## 1. Data Loading and Preprocessing.

In these steps, we will load the Bengaluru House Data dataset using pandas and perform an initial exploration to understand its structure and contents.

- **Data Import:** The dataset is loaded into a pandas DataFrame named `data`.
- **Shape:** The dataset contains 12,530 rows and 7 columns after initial cleaning.
- **Columns:**  
    - `location`: Area or locality of the property  
    - `size`: Number of bedrooms (e.g., "2 BHK", "4 Bedroom")  
    - `total_sqft`: Total area in square feet  
    - `bath`: Number of bathrooms  
    - `price`: Price of the property (in lakhs)  
    - `bhk`: Extracted number of bedrooms as integer  
    - `price_per_sqft`: Price per square foot

We will also check for missing values, data types, and unique values in key columns to guide further cleaning and preprocessing steps. This foundational understanding helps in identifying potential issues such as outliers, inconsistent data, and the need for encoding categorical variables.

In [2]:
import pandas as pd
import numpy as np

In [3]:
# import the data 
data = pd.read_csv('clean_data_for_EDA.csv')
data

,area_type,availability,location,total_sqft,bath,balcony,price,bhk
0,Super built-up Area,19-Dec,Electronic City Phase II,1056.0,2.0,1.0,39.07,2
1,Plot Area,Ready To Move,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4
2,Built-up Area,Ready To Move,Uttarahalli,1440.0,2.0,3.0,62.00,3
3,Super built-up Area,Ready To Move,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3
4,Super built-up Area,Ready To Move,Kothanur,1200.0,2.0,1.0,51.00,2
...,...,...,...,...,...,...,...,...
12700,Super built-up Area,Ready To Move,Green Glen Layout,1715.0,3.0,3.0,112.00,3
12701,Built-up Area,Ready To Move,Whitefield,3453.0,4.0,0.0,231.00,5
12702,Super built-up Area,Ready To Move,Richards Town,3600.0,5.0,2.0,400.00,4
12703,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,1141.0,2.0,1.0,60.00,2


In [4]:
data.head()

,area_type,availability,location,total_sqft,bath,balcony,price,bhk
0,Super built-up Area,19-Dec,Electronic City Phase II,1056.0,2.0,1.0,39.07,2
1,Plot Area,Ready To Move,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4
2,Built-up Area,Ready To Move,Uttarahalli,1440.0,2.0,3.0,62.00,3
3,Super built-up Area,Ready To Move,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3
4,Super built-up Area,Ready To Move,Kothanur,1200.0,2.0,1.0,51.00,2



### 2. Perform EDA For machine Learning
- Understand the business and business problem and data 
- data cleaning
- understand the data distributions
- outliers checks
- analyze the uni variant - bi-variant and multi variant  for check the data relation ship
- check pattern in data 
- change catgorical data into numerical data
- data scaling/feature Scaling
- feature engineering / 
- feature selection 

Checking for outliers.


In [ ]:
data[data.bhk>20]

only two columns that  have value greter than 20 so they are outliers.

Now let's check for `total_sqft` column.

In [ ]:
data[['price', 'total_sqft']].corr()

### Creating the `price_per_sqft` Feature

A new column, `price_per_sqft`, is added to represent the price per square foot for each property. This feature enables more effective comparison of property values across different locations and sizes.

In [ ]:
data['price_per_sqft'] = data['price']*100000 / data['total_sqft']

In [ ]:
data['price_per_sqft']

In [ ]:
data.describe()

In [ ]:
data['location'].value_counts()

`location` columns preprocessing we will remove outlier if any.

In [ ]:
# Remove leading and trailing spaces from location names
data['location'] = data['location'].apply(lambda x : x.strip())
location_count = data['location'].value_counts()


In [ ]:
location_count  

In [ ]:
location_count_less_10 = location_count[location_count <= 10]
location_count_less_10

Locations with 10 or fewer occurrences (totaling 1,053 unique locations) are replaced with `'other'`. This reduces the number of unique categories in the `location` column, simplifying encoding and improving model performance.

In [ ]:

data['location'] = data['location'].apply(lambda x: 'other' if x in location_count_less_10 else x)

In [ ]:
data['location'].value_counts()

In [ ]:
data.describe()

Outlier Detection in `total_sqft`

A review of the `total_sqft` column reveals some unrealistic values, such as properties with as little as 1 square foot of area. Such entries are clear outliers or data entry errors, as it is not feasible for any property—especially those with multiple bedrooms—to have such a small area.

To improve data quality and ensure reliable analysis, we remove properties where the average area per BHK (i.e., `total_sqft` divided by `bhk`) is less than 300 sqft. This threshold helps filter out likely outliers and data inconsistencies, resulting in a more accurate and trustworthy dataset for further exploration and modeling.

In [ ]:
data = data[((data['total_sqft']/data['bhk']) >= 300)]
data.describe()

In [ ]:
data.shape

In [ ]:
def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key,subdf in df.groupby('location'):
        
        m = np.mean(subdf['price_per_sqft'])

        st = np.std(subdf['price_per_sqft'])

        gen_df = subdf[(subdf['price_per_sqft'] > (m-st)) & (subdf['price_per_sqft'] <= (m+st))]
        
        df_output = pd.concat([df_output,gen_df],ignore_index= True)
       
    return df_output
data = remove_outliers_sqft(data)
data.describe()

In [ ]:
data.shape

### Understanding `groupby` and Outlier Removal Logic

- **`groupby('location')`** splits the DataFrame into groups based on unique values in the `location` column.
    - **`key`**: Stores the name of the current location (a string, e.g., `'Whitefield'`).
    - **`subdf`**: Contains a DataFrame with all rows for that location.

- For each location, the function keeps only those rows where `price_per_sqft` is within one standard deviation of the mean for that location. This filters out unusually high or low property prices, keeping only the most typical values for each area.


In [ ]:


def bhk_outlier_remove(df):
    exclude_indices = np.array([])

    for location, location_df in df.groupby('location'):
        bhk_stats = {}

        # Step 1: Calculate mean and std for each BHK in a location
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df['price_per_sqft']),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
  
        # Step 2: Compare each BHK price_per_sqft with the (BHK-1) stats
        
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk - 1)
            if stats and stats['count'] > 5:
                exclude_indices = np.append(
                    exclude_indices,
                    bhk_df[bhk_df['price_per_sqft'] < stats['mean']].index.values
                )

    return df.drop(exclude_indices, axis='index')


### Understanding and Removing BHK Outliers

The `bhk_outlier_remove` function refines real estate data by removing properties where the `price_per_sqft` is unusually low compared to properties with fewer rooms (BHK) in the same location. For example, it addresses cases where a 3BHK is listed at a lower price per square foot than a 2BHK nearby, which is typically unrealistic.

#### How It Works: Two-Step Process Per Location

**1. Calculate BHK Statistics**  
For each unique location, properties are grouped by their BHK count. For every BHK group within a location, the function computes:
- Mean `price_per_sqft`
- Standard deviation of `price_per_sqft`
- Count of properties for that BHK type

This builds a statistical profile for each BHK size in every location.

**2. Compare and Exclude Outliers**  
For each BHK type in a location:
- The function looks up the mean `price_per_sqft` of the (BHK - 1) group (e.g., for 3BHK, it checks 2BHK stats).
- If a property's `price_per_sqft` is less than the mean of the smaller BHK group (and the smaller group has more than 5 properties), it is flagged as an outlier.
- The indices of these outlier properties are collected.

**Final Step: Clean the DataFrame**  
After processing all locations and BHK groups, the function removes all identified outlier rows, resulting in a cleaner dataset for further analysis and modeling.


In [ ]:
data = bhk_outlier_remove(data)

In [ ]:
data.shape

In [ ]:
data

In [ ]:
data.drop(columns=['size','price_per_sqft'],inplace=True)

### Cleaned data

In [ ]:
data.head()

In [ ]:
data.to_csv("Cleaned_data.csv")

In [ ]:
X = data.drop(columns=['price'])
y = data['price']


## Model Building 

The necessary libraries for model building and evaluation have been imported, including tools for data preprocessing, regression algorithms (Linear Regression, Lasso, Ridge), and performance metrics. This sets the stage for splitting the data, encoding categorical features, scaling, and training regression models.
```

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression , Lasso,Ridge
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score


### Train-Test Split

The dataset is split into training and testing sets using an 80-20 ratio. This ensures that model evaluation is performed on unseen data, providing a reliable measure of performance.


In [ ]:
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state =0)

In [ ]:
print(X_train.shape)
print(X_test.shape)

### Applying Linear Regression

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

# Create column transformer
column_trans = make_column_transformer(
    (OneHotEncoder(sparse_output=False), ['location']),
    remainder='passthrough'
)


The `column_trans` object is a column transformer that applies one-hot encoding to the `location` column, converting categorical location values into binary features. All other columns are left unchanged (`remainder='passthrough'`). This prepares the data for machine learning models that require numerical input.


In [ ]:
scaler = StandardScaler()
lr = LinearRegression() 

In [ ]:
pipe = make_pipeline(column_trans, scaler, lr)

The `pipe` object is a machine learning pipeline that sequentially applies the following steps:

1. **`column_trans`**: Transforms the `location` column using one-hot encoding and passes through other features unchanged.
2. **`scaler`**: Standardizes numerical features to have zero mean and unit variance.
3. **`lr`**: Fits a Linear Regression model to the processed data.

This pipeline streamlines preprocessing and model training into a single, reusable workflow.

In [ ]:
pipe.fit(X_train, y_train)

Fits the pipeline to the training data, performing preprocessing and training the Linear Regression model.

In [ ]:
y_pred_lr = pipe.predict(X_test)

Performs predictions on the test set using the trained Linear Regression pipeline, generating estimated property prices for evaluation.

In [ ]:
r2 = r2_score(y_test, y_pred_lr)
print("R² Score:", r2)

### Applying Lasso

In [ ]:
lasso = Lasso()

In [ ]:

pipe = make_pipeline(column_trans, scaler,lasso)

Creates a pipeline with column transformer, scaler, and Lasso regression for modeling.

In [ ]:
pipe.fit(X_train,y_train)

Fits the pipeline to the training data, performing preprocessing and training the Linear Regression model.


Predicts property prices using the Lasso regression pipeline and evaluates model performance with the R² score.


In [ ]:
y_pred_lasso = pipe.predict(X_test)
r2_score(y_test,y_pred_lasso)

### Applying Ridge

In [ ]:
ridge = Ridge()

In [ ]:
pipe = make_pipeline(column_trans,scaler,ridge)

Creates a pipeline with column transformer, scaler, and Ridge regression for modeling.

In [ ]:
pipe.fit(X_train,y_train)

Fits the pipeline to the training data, performing preprocessing and training the Ridge Regression model.

Evaluating the Ridge regression model on the test set using the R² score.

In [ ]:
y_pred_ridge = pipe.predict(X_test)
r2_score(y_test,y_pred_ridge)

In [ ]:
print("No Regularisation :" ,r2_score(y_test,y_pred_lr))
print("Lasso :" ,r2_score(y_test,y_pred_lasso))
print("Ridge :" ,r2_score(y_test,y_pred_ridge))


# All Regression Models Apply

In [ ]:
df= pd.read_csv("Bengaluru_House_Data.csv")
df

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline

# Regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor

# ------------------------
# Step 1: Load Data
# ------------------------
data = {
    "location": ["1st Block Jayanagar"] * 10,
    "total_sqft": [2850.0, 1630.0, 1875.0, 1200.0, 1235.0,2850.0, 1630.0, 1875.0, 1200.0, 1235.0],
    "bath": [4.0, 3.0, 2.0, 2.0, 2.0,4.0, 3.0, 2.0, 2.0, 2.0],
    "price": [428.0, 194.0, 235.0, 130.0, 148.0,428.0, 194.0, 235.0, 130.0, 148.0],
    "bhk": [4, 3, 3, 3, 2,4, 3, 3, 3, 2]
}

df = pd.DataFrame(data)

# ------------------------
# Step 2: Prepare Features & Target
# ------------------------
X = df[['total_sqft', 'bath', 'bhk']]
y = df['price']

# Optional: Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data (though it's a small dataset)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ------------------------
# Step 3: Define Models
# ------------------------
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1),
    "ElasticNet": ElasticNet(alpha=1.0, l1_ratio=0.5),
    "Polynomial Regression (Degree=2)": make_pipeline(PolynomialFeatures(degree=2), LinearRegression()),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(n_estimators=100),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Support Vector Regressor": SVR(),
    "Neural Network (MLP)": MLPRegressor(hidden_layer_sizes=(64, 64), max_iter=1000),
    "KNN Regressor": KNeighborsRegressor(n_neighbors=2)
}

# ------------------------
# Step 4: Train and Evaluate
# ------------------------
print("\n📊 Regression Model Performance (MSE on Test Set):\n")

for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print(f"{name}: MSE = {mse:.2f} : R2_Score ={r2}")


## Model Comparison and Final Selection

The R² scores for the different regression models are as follows:

- **Linear Regression (No Regularisation):** 0.827
- **Lasso Regression:** 0.813
- **Ridge Regression:** 0.828

Both Linear and Ridge regression models perform similarly and outperform Lasso regression. Given Ridge regression's ability to handle multicollinearity and prevent overfitting through regularization, we will proceed with **Ridge Regression** as our final model for property price prediction.

In [ ]:
# train_model_and_save.py

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
import joblib

# Load and clean data
df = pd.read_csv("Cleaned_data.csv")
if "Unnamed: 0" in df.columns:
    df.drop(columns=["Unnamed: 0"], inplace=True)

X = df.drop(columns=["price"])
y = df["price"]

# Create preprocessing + model pipeline
column_trans = make_column_transformer(
    (OneHotEncoder(sparse_output=False, handle_unknown="ignore"), ["location"]),
    remainder="passthrough"
)
pipeline = make_pipeline(column_trans, StandardScaler(), Ridge())

# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# Save model and location list
joblib.dump(pipeline, "bengaluru_house_model.pkl")
joblib.dump(X["location"].unique().tolist(), "location_list.pkl")
print("✅ Model and location list saved successfully.")


## Streamlit app

In [ ]:
#  c:\Users\hp\Documents\ds\ds_materials\8.1_streamlit\streamlit_tutorial_in_1_class.ipynb

## using save model

In [ ]:
# bengaluru_app.py

import streamlit as st
import pandas as pd
import numpy as np
import joblib

# Load model and location list
model = joblib.load("bengaluru_house_model.pkl")
locations = joblib.load("location_list.pkl")

# Streamlit UI
st.set_page_config(page_title="Bengaluru House Price Predictor", layout="centered")
st.title("🏠 Bengaluru House Price Predictor")

st.sidebar.header("Input House Details")

location = st.sidebar.selectbox("Select Location", sorted(locations))
total_sqft = st.sidebar.number_input("Total Square Feet", min_value=300.0, max_value=10000.0, value=1000.0)
bhk = st.sidebar.slider("Bedrooms (BHK)", 1, 10, 2)
bath = st.sidebar.slider("Bathrooms", 1, 10, 2)

if st.sidebar.button("Predict Price"):
    input_data = pd.DataFrame({
        "location": [location],
        "total_sqft": [total_sqft],
        "bhk": [bhk],
        "bath": [bath]
    })
    
    prediction = model.predict(input_data)[0]
    st.success(f"💰 Estimated Price: ₹{prediction:.2f} Lakhs")

st.markdown("---")
st.caption("Built with ❤️ using Streamlit and Scikit-Learn")


## 🟢 How to Run This App

1. Run model training:

   ```bash
   python train_model_and_save.py
   ```

2. Start Streamlit app:

   ```bash
   streamlit run bengaluru_app.py
   ```

---

## 📦 Optional: `requirements.txt`

```txt
streamlit
pandas
scikit-learn
joblib
```

---

## Without model save 

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
import joblib

# Load cleaned data
data = pd.read_csv("Cleaned_data.csv")
data.drop(columns=["Unnamed: 0"], inplace=True)

# Prepare features and target
X = data.drop(columns=["price"])
y = data["price"]

# Create column transformer
column_trans = make_column_transformer(
    (OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ['location']),
    remainder='passthrough'
)

# Create Ridge pipeline
scaler = StandardScaler()
ridge = Ridge()
pipe = make_pipeline(column_trans, scaler, ridge)

# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
pipe.fit(X_train, y_train)

# Streamlit App
st.title("Bengaluru House Price Predictor")

# Sidebar inputs
st.sidebar.header("User Input Features")
location = st.sidebar.selectbox("Location", sorted(data["location"].unique()))
total_sqft = st.sidebar.number_input("Total Sqft Area", min_value=300.0, max_value=10000.0, value=1000.0)
bhk = st.sidebar.slider("Number of Bedrooms (BHK)", 1, 10, 2)
bath = st.sidebar.slider("Number of Bathrooms", 1, 10, 2)

# Predict button
if st.sidebar.button("Predict Price"):
    input_df = pd.DataFrame({
        'location': [location],
        'total_sqft': [total_sqft],
        'bath': [bath],
        'bhk': [bhk]
    })
    prediction = pipe.predict(input_df)[0]
    st.success(f"Predicted House Price: ₹{round(prediction, 2)} Lakhs")

# Dataset preview
if st.checkbox("Show Cleaned Dataset"):
    st.dataframe(data.head(20))

# Model performance
st.markdown("---")
st.subheader("Model Evaluation")
r2_score = pipe.score(X_test, y_test)
st.write(f"R² Score on Test Set: {r2_score:.4f}")
